In [1]:
import random
from pathlib import Path
import spacy
from spacy.training.example import Example
from spacy.pipeline.tagger import Tagger
nlp = spacy.load("en_core_web_sm")

## How to update a dependency parser starting off with a blank model

## How to train our parsers to understand new semantic relationships or dependencies between words.

In [2]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# training data: texts, heads and dependency labels
# for no relation, we simply chose an arbitrary dependency label, e.g. '-'
TRAIN_DATA = [
    (
        "find some cafes with great wifi",
        {
            "heads": [0, 2, 0, 5, 5, 2],  # index of token head
            "deps": ["ROOT", "QUANTITY", "PLACE", "-", "QUALITY", "ATTRIBUTE"],
        },
    ),
    (
        "find a few hotel near the beach",
        {
            "heads": [0, 3, 3, 0, 5, 5, 2],
            "deps": ["ROOT", "-", "QUANTITY", "PLACE", "QUALITY", "-", "ATTRIBUTE"],
        },
    ),
    (
        "find me numerous gyms that are open late",
        {
            "heads": [0, 0, 3, 0, 5, 3, 5, 5],
            "deps": ["ROOT","-","QUANTITY","PLACE","-","-","ATTRIBUTE","TIME"],
        },
    ),
    (
        "show me many stores that sell flowers",
        {
            "heads": [0, 0, 3, 0, 3, 3, 3],  # attach "flowers" to store!
            "deps": ["ROOT", "-", "QUANTITY", "PLACE", "-", "-", "PRODUCT"],
        },
    ),
    (
        "walk a whole day in london",
        {
            "heads": [0, 3, 3, 0, 3, 3],
            "deps": ["ROOT", "-", "QUANTITY", "TIME", "-", "LOCATION"],
        },
    ),
    (
        "walk enough hours in london",
        {
            "heads": [0, 2, 0, 2, 2],
            "deps": ["ROOT", "QUANTITY", "TIME", "-", "LOCATION"],
        },
    ),
    (
        "show me all the hostels in berlin",
        {
            "heads": [0, 0, 4, 4, 0, 4, 4],
            "deps": ["ROOT", "-", "QUANTITY", "-", "PLACE", "-", "LOCATION"],
        },
    ),
    (
        "give me half the money in your pocket",
        {
            "heads": [0, 0, 4, 4, 0, 7, 7, 4],
            "deps": ["ROOT","-","QUANTITY","-","ATTRIBUTE","-","-","LOCATION"],
        },
    ),
]


# @plac.annotations(
#     model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#     output_dir=("Optional output directory", "option", "o", Path),
#     n_iter=("Number of training iterations", "option", "n", int),
# )
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the parser."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # We'll use the built-in dependency parser class, but we want to create a
    # fresh instance – just in case.
    if "parser" in nlp.pipe_names:
        nlp.remove_pipe("parser")
    # parser = nlp.create_pipe("parser")
    parser = nlp.add_pipe('parser', first=True)

    for text, annotations in TRAIN_DATA:
        for dep in annotations.get("deps", []):
            parser.add_label(dep)

    pipe_exceptions = ["parser", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train parser
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            for batch in minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
                examples = []
                for text, annotations in batch:
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    examples.append(example)
                nlp.update(examples, drop=0.5, losses=losses)
            # print("Losses", losses)

    # test the trained model
    test_model(nlp)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        test_model(nlp2)


def test_model(nlp):
    texts = [
        "find enough hotels with good wifi",
        "find some gyms near work",
        "show me many hotels in berlin",
    ]
    docs = nlp.pipe(texts)
    for doc in docs:
        print(doc.text)
        print([(t.text, t.dep_, t.head.text) for t in doc if t.dep_ != "-"])


if __name__ == "__main__":
    main()


Created blank 'en' model
find enough hotels with good wifi
[('find', 'ROOT', 'find'), ('enough', 'QUANTITY', 'hotels'), ('hotels', 'PLACE', 'find'), ('good', 'QUALITY', 'wifi'), ('wifi', 'ATTRIBUTE', 'hotels')]
find some gyms near work
[('find', 'ROOT', 'find'), ('some', 'QUANTITY', 'gyms'), ('gyms', 'PLACE', 'find'), ('near', 'QUALITY', 'work'), ('work', 'ATTRIBUTE', 'gyms')]
show me many hotels in berlin
[('show', 'ROOT', 'show'), ('many', 'QUANTITY', 'hotels'), ('hotels', 'PLACE', 'show'), ('berlin', 'LOCATION', 'hotels')]
